# Часть 1. Сегментация

### Задача

Обучите нейронную сеть сегментировать границы клеток.

В этой задаче вам не будут предоставлены какие-либо фрагменты кода, только входные данные и целевая метрика - intersection-over-union (IoU).

Вы должны обучить свою нейронную сеть предсказывать маску краевых пикселей (пикселей в ground truth изображениях со значением больше 0).

Используйте все, что вы узнали на данный момент:
* любые архитектуры для семантической сегментации
* аугментации (понадобятся, так как в трейн выборке всего 41 изображение)
* fine-tuning

Не разрешается только одна вещь: тренировать сеть на тест сете.

Финальное решение будет состоять из jupyter ноутбука с кодом (для обучения сети + любые эксперименты с данными) и архива с изображениями png с предсказаниями сети для тестовых изображений (одноканальные изображения, 0 - для некраевых пикселей, любое ненулевое значение для краевых пикселей).

Хорошая сеть должна уметь сегментировать изображения с iou >= 0.29. Это не строгий критерий, но попытайтесь получить аналогичные или лучшие числа.

Практические заметки:
* В датасете присутствует сильный дисбаланс классов, поэтому предсказания будут "смещены" в сторону "нулевого" класса. Вы можете либо настроить минимальный порог вероятности для класса «граница», либо добавить вес классу в лосс функции.
* Датасет - маленький, активно используйте аугментации: вращение, отражение, случайный контраст и яркость
* Лучше потратье время на эксперименты с нейронной сетью, чем на постпроцессинг (т.е. test-time augmentation).
* Имейте в виду, что архитектура сети определяет receptive field пикселей на выходе. Если размер картинки на вхож меньше, чем receptive field пикселя на выходе, вы можете попробовать "выкинуть" сколько-то слоёв из сети без потери качества. Вполне нормально изменять "готовые" (of-the-shelf) архитектуры

In [ ]:
### Download the dataset ###
!wget https://www.dropbox.com/s/jy34yowcf85ydba/data.zip?dl=0 -O data.zip
!unzip -q data.zip

In [ ]:
### Визуализируем данные ###
import matplotlib.pyplot as plt
import numpy as np
import skimage
from skimage import io
%matplotlib inline

# Human HT29 colon-cancer cells
plt.figure(figsize=(10,8))
plt.subplot(1,2,1)
im = skimage.img_as_ubyte(io.imread('BBBC018_v1_images-fixed/train/00735-actin.DIB.bmp'))
plt.imshow(im)
plt.subplot(1,2,2)
mask = skimage.img_as_ubyte(io.imread('BBBC018_v1_outlines/train/00735-cells.png'))
plt.imshow(mask, 'gray')

In [ ]:
### Метрика ###
def calc_iou(prediction, ground_truth):
    n_images = len(prediction)
    intersection, union = 0, 0
    for i in range(n_images):
        intersection += np.logical_and(prediction[i] > 0, ground_truth[i] > 0).astype(np.float32).sum() 
        union += np.logical_or(prediction[i] > 0, ground_truth[i] > 0).astype(np.float32).sum()
    return float(intersection) / union

In [ ]:
# ваш код здесь

# Часть 3. Deep Image Prior (DIP)


### Основная идея

Идея статьи состоит в том, что сверточные сети задают хороший "prior" на изображения. Подробнее в [1].

Представьте, что у вас есть "испорченное" изображение $x_0$ и оригинальное хорошее изображение $x$.

$x_0$ может представлять зашумленное изображение (задача denoising), либо часть изображения отсутствует (задача inpainting), либо например изображение маленького размера (задача super-resolution)

Зададим $x$ следующим образом:

$x = f_\theta(\mathcal{z})$

где $f$ - нейронная сеть, $z$ - фиксированный input

Будем восстанавливать $x$ с помощью следующей оптимизационной процедуры:

1. Задаем и фиксируем $z$, как тензор состоящий например из равномерного шума, высотой и шириной как изображение $x$, но с числом каналов, например 32
2. Инциализиурем веса сети $\theta$
3. Обновляем веса сети с помощью оптимизации функции ошибки $E$:

$$
\min_\theta E(f_\theta(z), x_0).
$$

4. Повторяем шаг 3 (процедуру оптимизации) $M$ раз, и останавливаем её
5. Смотрим на результат: $f_\theta(\mathcal{z})$


### Задача

Главная цель - вопроизвести результаты статьи [1].

1. выполните denoising изображений в папке "data/denoising" ($x_0 = x + \text{гауссовский шум}$),
2. выполните inpainting изображений в папке "data/inpaiting", (примените лосс только на известный участок изображения, используя предоставленную маску)  ($x_0 = x \text{ c вырезанной маской}$)

используя Mean Square Error (MSE) как функцию ошибки $E$,

с помощью UNet-подобной сети (4 downsampling слоя, и 16, 32, 64, 128 фильтра на выходе каждого блока соответственно).

Как сделать подобную архитектуру:
https://towardsdatascience.com/unet-line-by-line-explanation-9b191c76baf5


### Примечания

- Поиграйтесь с числом итераций $M$, используемых для early stopping, и найдите оптимальные значения для denoising и inpainting задач. Код и результаты должны быть представлены в этом ноутбуке.
- Используйте Adam как оптимизатор по умолчанию, но можете спокойно воспользоваться любым другим и даже можете попробовать другую архитектуру сети.

### Ссылки

[1] Ulyanov et. al., "Deep Image Prior", CVPR 2018, https://arxiv.org/abs/1711.10925

In [ ]:
### Download the dataset ###
!wget https://www.dropbox.com/s/si5o4dp4qa59cyy/data.zip?dl=0 -O data.zip
!unzip -q data.zip

In [ ]:
# YOUR CODE HERE

# Часть 2. Variational Autoencoder (VAE) .

Оригинальная статья http://arxiv.org/abs/1312.6114

В этой части мы обучим автоенкодер и вариационный автоенкодер на датасете "Labeled Faces in the Wild" dataset (LFW) (http://vis-www.cs.umass.edu/lfw/).

### Подготовьте данные

In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!wget https://raw.githubusercontent.com/yandexdataschool/Practical_DL/hw3_19/homework03/lfw_dataset.py -O lfw_dataset.py

In [ ]:
#@title Utility functions
import numpy as np
import os
import skimage.io
import skimage
import skimage.transform
import pandas as pd

def fetch_lfw_dataset(attrs_name = "lfw_attributes.txt",
                      images_name = "lfw-deepfunneled",
                      raw_images_name = "lfw",
                      use_raw=False,
                      dx=80,dy=80,
                      dimx=45,dimy=45
    ): # sad smile

    #download if not exists
    if (not use_raw) and not os.path.exists(images_name):
        print("images not found, donwloading...")
        os.system("wget http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz -O tmp.tgz")
        print("extracting...")
        os.system("tar xvzf tmp.tgz && rm tmp.tgz")
        print("done")
        assert os.path.exists(images_name)
    
    if use_raw and not os.path.exists(raw_images_name):
        print("images not found, donwloading...")
        os.system("wget http://vis-www.cs.umass.edu/lfw/lfw.tgz -O tmp.tgz")
        print("extracting...")
        os.system("tar xvzf tmp.tgz && rm tmp.tgz")
        print("done")
        assert os.path.exists(raw_images_name)

    if not os.path.exists(attrs_name):
        print("attributes not found, downloading...")
        os.system("wget http://www.cs.columbia.edu/CAVE/databases/pubfig/download/%s" % attrs_name)
        print("done")

    #read attrs
    df_attrs = pd.read_csv("lfw_attributes.txt",sep='\t',skiprows=1,) 
    df_attrs = pd.DataFrame(df_attrs.iloc[:,:-1].values, columns = df_attrs.columns[1:])


    #read photos
    dirname = raw_images_name if use_raw else images_name
    photo_ids = []
    for dirpath, dirnames, filenames in os.walk(dirname):
        for fname in filenames:
            if fname.endswith(".jpg"):
                fpath = os.path.join(dirpath,fname)
                photo_id = fname[:-4].replace('_',' ').split()
                person_id = ' '.join(photo_id[:-1])
                photo_number = int(photo_id[-1])
                photo_ids.append({'person':person_id,'imagenum':photo_number,'photo_path':fpath})

    photo_ids = pd.DataFrame(photo_ids)

    #mass-merge
    #(photos now have same order as attributes)
    df_attrs['imagenum'] = df_attrs['imagenum'].astype(np.int64)
    df = pd.merge(df_attrs, photo_ids, on=('person','imagenum'))

    assert len(df)==len(df_attrs),"lost some data when merging dataframes"

    #image preprocessing
    all_photos = df['photo_path'].apply(lambda img: skimage.io.imread(img))\
                                 .apply(lambda img:img[dy:-dy,dx:-dx])\
                                 .apply(lambda img: skimage.img_as_ubyte(skimage.transform.resize(img,[dimx,dimy])))

    all_photos = np.stack(all_photos.values).astype('uint8')
    all_attrs = df.drop(["photo_path","person","imagenum"],axis=1)
    
    return all_photos,all_attrs

In [ ]:
data, attrs = fetch_lfw_dataset(dimx=36,dimy=36)

In [ ]:
data = data/255
np.savez("real.npz", Pictures=data.reshape(data.shape[0], 36*36*3))

In [ ]:
X_train = data[:10000].reshape((10000, -1))
print(X_train.shape)
X_val = data[10000:].reshape((-1, X_train.shape[1]))
print(X_val.shape)

image_h = data.shape[1]
image_w = data.shape[2]

For simplicity we want all values of the data to lie in the interval $[0,1]$:

In [ ]:
X_train = np.float32(X_train)
X_val = np.float32(X_val)

In [ ]:
def plot_gallery(images, h, w, n_row=3, n_col=6):
    """Helper function to plot a gallery of portraits"""
    plt.figure(figsize=(1.5 * n_col, 1.7 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w, 3)), cmap=plt.cm.gray, vmin=-1, vmax=1, interpolation='nearest')
        plt.xticks(())
        plt.yticks(())

In [ ]:
plot_gallery(X_train, image_h, image_w)

In [ ]:
train = data_utils.TensorDataset(torch.Tensor(X_train), torch.zeros(X_train.shape[0],)) # pseudo labels needed to define TensorDataset
train_loader = data_utils.DataLoader(train, batch_size=100, shuffle=True)

val = data_utils.TensorDataset(torch.Tensor(X_val), torch.zeros(X_val.shape[0],))
val_loader = data_utils.DataLoader(val, batch_size=1, shuffle=False)

## Autoencoder

В чем суть вариационного автоенкодера со всеми сложными формулами и регуляризациями? Чтобы ощутить разницу обучите сначала автоенкодер:

<img src="https://lilianweng.github.io/lil-log/assets/images/autoencoder-architecture.png" alt="Autoencoder">

In [ ]:
dimZ = 100 # Учитывая, что задача реконструкции лица, какой размер скрытого представления кажется разумным?

# Определите декодер и енкодер как сети с 1-м скрытым полносвязным слоем
# (это будет означать что в каждой сети будет 2 полносвзяных слоя)
# Используйте ReLU на активациях скрытых слоёв
# GlorotUniform инициализация для W
# Zero инициализация для biases
# Удобно добавить sigmoid активацию на выход сети, чтобы получить нормализованный output

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        #TODO
        
        # self.encoder = 
        # self.decoder =
        
    def forward(self, x):
        
        #TODO
        
        # latent_code = 
        # reconstruction = 
        
        return reconstruction, latent_code

In [ ]:
# Создаём MSE loss function
criterion = torch.nn.MSELoss()

autoencoder = Autoencoder().cuda()

# используем Adam оптимизиатор
optimizer = optim.Adam(autoencoder.parameters())

In [ ]:
# Обучите autoencoder
# Отобразите прогресс реконструкции изображения и падение лосса

In [ ]:
# Отобразите реконструкции
for j, data in enumerate(val_loader, 0):
    inp = data[0].cuda()
    pred, _ = autoencoder(inp)
    plot_gallery([data[0].numpy(), pred.data.cpu().numpy()], image_h, image_w, n_row=1, n_col=2)
    if (j >= 9):
        break

Реконструкция не так уж и плоха, да?

## Sampling

In [ ]:
for i, (putin, y) in enumerate(val_loader):
    if i == 2754:
        break
plt.imshow(putin.numpy().reshape((image_w, image_w, 3)))

In [ ]:
plt.figure(figsize=(10, 12))
plt.suptitle('Twin farm')
for i in range(len(image_progress[:20])):
    plt.subplots_adjust(bottom=0.0, left=.1, right=.9, top=.50, hspace=.15)
    plt.subplot(6, 5, 5*(i//5) + i % 5 + 1)
    plt.imshow(image_progress[i].clamp(0,1).data.cpu().numpy().reshape(image_w, image_h, 3))
    plt.title('Epoch = {}'.format(i * 5 + 1))
    plt.axis('off')
plt.tight_layout()

Давайте насемплим несколько случайных латентных векторов $z$ и сделаем inference - реконструируем изображения из $z$

In [ ]:
z = (np.random.randn(25, dimZ)*0.5).astype('float32')
output = autoencoder.decoder(torch.from_numpy(z).cuda()).clamp(0, 1)
plot_gallery(output.data.cpu().numpy(), image_h, image_w, n_row=5, n_col=5)

Если мы будем семплить $z$ из нормального распределения, мы в итоге получим на выходе все возможные лица? Как вы считаете?

## Variational Autoencoder

Байесовский подход в глубоком обучении рассматривает всё в терминах распределений. Теперь наш енкодер будет генерировать не просто $z$, но и апостериорное распределение $q(z|x)$. В нашем случае распределение $q$ является гауссовским распределением $N(\mu, \sigma)$ с параметрами $\mu$, $\sigma$. Технически, первое отличие заключается в том, что вам нужно разделить bottleneck слой на два слоя. Один полносвязный слой будет генерировать вектор $\mu$, а другой - вектор $\sigma$. Reparametrization trick должен быть реализован с помощью **gaussian_sampler** слоя, который генерирует случайный вектор $\epsilon$ и возвращает $z=\mu+\sigma\epsilon \sim N(\mu, \sigma)$.

Поскольку наш декодер также является функцией, которая генерирует распределение, нам необходимо выполнить такое же разбиение для выходного слоя. При тестировании модели мы будем смотреть только на средние значения, поэтому одной частью выходных данных будет фактический выходной сигнал автоэнкодера.

Реализуйте простейшую версию VAE - один $z$ на вход. Можно также попробовать рассмотреть выборку из нескольких $z$  на один вход и далее усреднить их.

In [ ]:
# чтобы сравниться с autoencoder выберите такой же dimZ как и до этого
dimZ = 100

# напишите сеть
# можно посмотреть на код семинара, либо https://github.com/pytorch/examples/blob/master/vae/main.py

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        #TODO
    
    def gaussian_sampler(self, mu, logsigma):
        if self.training:
            std = logsigma.exp()
            eps = std.data.new(std.size()).normal_()
            return eps.mul(std).add(mu)
        else:
            return mu

    def forward(self, x):
        
        #TODO
        # return reconstruction_mu, reconstruction_logsigma, latent_mu, latent_logsigma

Функционал, который будем оптимизировать для VAE имеет собственное название - variational lowerbound. Мы будем его максимизировать. Вот он (один $z$ на вход $x$):

$$\mathcal{L} = -D_{KL}(q_{\phi}(z|x)||p(z)) + \log p_{\theta}(x|z)$$

Реализуйте две функции, одна из которых будет считать KL-дивергенцию, а другая log-likelihood вашего output. Вот необходимая математика для удобства:

$$D_{KL} = -\frac{1}{2}\sum_{i=1}^{dimZ}(1+log(\sigma_i^2)-\mu_i^2-\sigma_i^2)$$
$$\log p_{\theta}(x|z) = \sum_{i=1}^{dimX}\log p_{\theta}(x_i|z)=\sum_{i=1}^{dimX} \log \Big( \frac{1}{\sigma_i\sqrt{2\pi}}e^{-\frac{(\mu_i-x)^2}{2\sigma_i^2}} \Big)=...$$

Не забывайте, что вы используете $\log\sigma$ на вход. Почему не просто $\sigma$?

In [ ]:
def KL_divergence(mu, logsigma):
    return 0

def log_likelihood(x, mu, logsigma):
    return 0

def loss_vae(x, mu_gen, logsigma_gen, mu_z, logsigma_z):
    return 0

Теперь учим модель:

In [ ]:
# обучите VAE
# Отобразите прогресс реконструкции изображения и падение лосса

In [ ]:
val_loader = data_utils.DataLoader(val, batch_size=1, shuffle=True)
vae.eval()
for j, data in enumerate(val_loader, 0):
    input = data[0].cuda()
    reconstruction_mu, _, _, _ = vae(input)
    plot_gallery([data[0].numpy(), reconstruction_mu.data.cpu().numpy()], image_h, image_w, n_row=1, n_col=2)
    if (j >= 9):
        break

И теперь насемплим с помощью VAE:

In [ ]:
# TODO
# Насэмплите (сгенерируйте) изображения из выученного распределения
# 1) Sample z ~ N(0,1)
# 2) Sample from N(decoder_mu(z), decoder_sigma(z))

Даже если на практике вы не видите большой разницы между AE и VAE, или VAE еще хуже, маленький "Байес" внутри вас должен прыгать от радости прямо сейчас.

В VAE вы можете по-настоящему семплить из распределения изображений $p(x)$, тогда как в AE нет простого и правильного способа сделать это.

## И напоследок!

Если вам удалось обучить свои автоэнкодеры и они что-то узнали о мире, то пришло время воспользоваться этим. Как вы могли заметить, в наборе данных есть атрибуты лица. Нас интересует параметр-колонка "Smiling", но можете поробовать и другие! Вот первая задача:

1) Извлеките атрибут "Smilling" и создайте два набора изображений: 10 улыбающихся и 10 не улыбающихся.

2) Вычислите скрытые (латентные) представления для каждого изображения в «улыбающемся» наборе и усредните эти вектора. Сделайте то же самое для "не улыбающегося" набора. Вы тем самым нашли **"векторное представление"** атрибутов "smile" и "no smile".

3) Вычислите разницу: вектор «улыбка» минус вектор «не улыбка».

3) Теперь проверьте, работает ли **«арифметика признаков (фичей)»**. Возьмите лицо без улыбки, закодируйте его с помощью екнодера, добавьте к нему разницу из пункта 3) и подавйте на вход декодеру. Проверьте работает ли в случае AE? в случае VAE?

In [ ]:
# TODO